In [3]:
import torch
import os
import sys

# Try to import the model. If this fails, make sure model.py is in the same folder.
try:
    from model import *
except ImportError:
    # Fallback/Placeholder if user hasn't created model.py yet
    pass

# ==========================================
# 1. CONFIGURATION
# ==========================================
# List your model filenames here
MODEL_FILES = [
    "best_model_CBAM_side_new_loss.pth",
    "best_model_CBAM_top_new_loss.pth",
    "best_model_CBAM_front_new_loss.pth"
]

# IMPORTANT: You must import your actual Model Class here!
# If you don't have the class definition, PyTorch cannot load the architecture.
# from my_model_file import MyCustomModel 

# FIX: Set this to the CLASS NAME, not an instance (do not add parentheses '()')
# If your model requires arguments, hardcode them in step 2 below.
try:
    MODEL_CLASS = CBAM_UNet  # Correct: The Class itself
except NameError:
    MODEL_CLASS = None       # Fallback if class isn't found
    print("Warning: CBAM_UNet class not found. Ensure 'from model import *' is working.")


# ==========================================
# 2. OPTIMIZATION FUNCTION
# ==========================================

def convert_to_half_precision(model_path):
    """
    Method 1: Float16 (Half Precision)
    Reduces size by 50%. Very safe for accuracy.
    """
    print(f"Processing (Float16): {model_path}...")
    
    try:
        # Load the model
        if MODEL_CLASS:
            # If loading from state_dict, we instantiate a NEW model here
            model = MODEL_CLASS() 
            model.load_state_dict(torch.load(model_path, map_location='cpu'))
        else:
            # If loading full model
            model = torch.load(model_path, map_location='cpu')
            
        # Convert to Half Precision
        model.half() 
        
        # Save
        new_path = model_path.replace(".pth", "_fp16.pth")
        torch.save(model, new_path)
        
        original_size = os.path.getsize(model_path) / (1024 * 1024)
        new_size = os.path.getsize(new_path) / (1024 * 1024)
        print(f" -> Saved to {new_path}")
        print(f" -> Size reduced from {original_size:.2f} MB to {new_size:.2f} MB")
        
    except Exception as e:
        print(f"Error processing {model_path}: {e}")
        import traceback
        traceback.print_exc()

# ==========================================
# 3. MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    print("--- Starting Float16 Optimization ---")
    for f in MODEL_FILES:
        if os.path.exists(f):
            convert_to_half_precision(f)
        else:
            print(f"File not found: {f}")

--- Starting Float16 Optimization ---
Processing (Float16): best_model_CBAM_side_new_loss.pth...


C:\Users\kiw00\AppData\Local\Temp\ipykernel_19756\2834455290.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location='

 -> Saved to best_model_CBAM_side_new_loss_fp16.pth
 -> Size reduced from 118.59 MB to 59.34 MB
Processing (Float16): best_model_CBAM_top_new_loss.pth...
 -> Saved to best_model_CBAM_top_new_loss_fp16.pth
 -> Size reduced from 118.59 MB to 59.34 MB
Processing (Float16): best_model_CBAM_front_new_loss.pth...
 -> Saved to best_model_CBAM_front_new_loss_fp16.pth
 -> Size reduced from 118.59 MB to 59.34 MB


In [4]:
device = 'cpu'  # AWS Free Tier uses CPU

def load_fp16_models():
    print("Loading Float16 models...")
    
    # NOTE: We use torch.load() directly because optimize_models.py 
    # saved the *entire* model object, not just the state_dict.
    try:
        model_side = torch.load('best_model_CBAM_side_new_loss_fp16.pth', map_location=device)
        model_top = torch.load('best_model_CBAM_top_new_loss_fp16.pth', map_location=device)
        model_front = torch.load('best_model_CBAM_front_new_loss_fp16.pth', map_location=device)
        
        # Ensure they are in eval mode
        model_side.eval()
        model_top.eval()
        model_front.eval()
        
        return model_side, model_top, model_front
    except Exception as e:
        print(f"Error loading models: {e}")
        return None, None, None

# Load them once (Global scope)
model_side, model_top, model_front = load_fp16_models()

Loading Float16 models...


C:\Users\kiw00\AppData\Local\Temp\ipykernel_19756\2863869479.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_side = torch.load('best_model_CBAM_side_new_loss_fp16.

In [13]:
import cv2
import numpy as np

In [14]:
img = cv2.imread('test_side.jpg')

In [15]:


input_arr = img[:,:,1]
temp_imput = (input_arr / 255.0) * 2 - 1
temp_imput = cv2.resize(temp_imput, (256,256))
temp_imput = np.expand_dims(temp_imput, axis=-1)
tensor = torch.from_numpy(temp_imput).permute(2, 0, 1).float().to(device).half()
model_side.eval()
with torch.no_grad():
    preds = model_side(tensor.unsqueeze(0))
result = np.squeeze(preds.cpu().numpy())
final_result = ((result + 1) / 2 * 255).astype(np.uint8)